In [1]:
import pandas as pd
import time

Import officer-involved fatality data from WaPo Github repo

In [29]:
fatalities = pd.read_csv('https://raw.githubusercontent.com/washingtonpost/data-police-shootings/master/fatal-police-shootings-data.csv')

Concatenate 'city' and 'state' values into a location name that can be geocoded

In [30]:
fatalities['location'] = fatalities[['city', 'state']].apply(lambda x: ', '.join(x), axis=1)

Read unique locations into a list

In [31]:
locations = []

for i in fatalities.location.unique():
    locations.append(i)

Convert to pandas series object

In [32]:
locations = pd.DataFrame(locations)

In [33]:
locations

,0
0,"Shelton, WA"
1,"Aloha, OR"
2,"Wichita, KS"
3,"San Francisco, CA"
4,"Evans, CO"
5,"Guthrie, OK"
6,"Chandler, AZ"
7,"Assaria, KS"
8,"Burlington, IA"
9,"Knoxville, PA"


Rename column

In [34]:
locations = locations.rename({0: "location"}, axis = 1)

In [36]:
locations.head()

,location
0,"Shelton, WA"
1,"Aloha, OR"
2,"Wichita, KS"
3,"San Francisco, CA"
4,"Evans, CO"


Export dataframe as input .csv for geocoding script; output will be 'fatalities_geocoded.csv'

In [7]:
locations.to_csv('locations.csv')

In [55]:
# importing csv module
import csv
import requests
import time

GOOGLE_MAPS_API_URL = 'https://maps.googleapis.com/maps/api/geocode/json?key=[YOUR API KEY HERE]'

infile = "locations.csv"
outfile = "fatalities_geocoded.csv"
rows = []

# reading csv file
with open(infile, 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)

    # extracting each data row one by one
    for row in csvreader:
        rows.append(row)

    # get total number of rows
    print("Total no. of rows: %d"%(csvreader.line_num))

for row in rows:
    address = row[-1]
    print("Processing: " + address)

    params = {
                'address': address,
                'sensor': 'false',
                'region': 'USA'
            }

    # Do the request and get the response data
    req = requests.get(GOOGLE_MAPS_API_URL, params=params)
    res = req.json()

    # Use the first result
    if(len(res['results']) > 0):

        result = res['results'][0]

        geodata = dict()
        geodata['lat'] = result['geometry']['location']['lat']
        geodata['lng'] = result['geometry']['location']['lng']

        # check if the coordinates were found
        if(geodata['lat'] is not None and geodata['lng'] is not None):
            row.append(geodata['lat'])
            row.append(geodata['lng'])

    else:
        print("Some problems with finding the coordinates. ", res);

#     time.sleep(2)


with open(outfile, 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(rows)

csvfile.close()
writeFile.close()


Total no. of rows: 63
Processing: location
Processing: Guthrie, OK
Processing: Burlington, IA
Processing: Columbus, OH
Processing: New Orleans, LA
Processing: South Gate, CA
Processing: Albuquerque, NM
Processing: Jourdanton, TX
Processing: Old Bridge, NJ
Processing: Fort Worth, TX
Processing: Hemet, CA
Processing: Austin, TX
Processing: Phoenix, AZ
Processing: Stillwater, OK
Processing: Tempe, AZ
Processing: Smyrna, ME
Processing: Kansas City, KS
Processing: Tarboro, NC
Processing: Medford, OR
Processing: Dublin, NC
Processing: Glendale, AZ
Processing: Harmony, IN
Processing: Islip, NY
Processing: Winchester, VA
Processing: Wilmore, KY
Processing: Sidney, NE
Processing: Kings County, CA
Processing: Marion, IL
Processing: Ft. Lauderdale, FL
Processing: Byram, NJ
Processing: Olivehurst, CA
Processing: Fort Collins, CO
Processing: Roswell, NM
Processing: Nitro, WV
Processing: Boerne, TX
Processing: West Chester, PA
Processing: Cushing, OK
Processing: Artesia, NM
Processing: Mitchell, SD


I fixed column headers in the .csv files manually in order to quickly join the two dataframes. The only reason I even had to join dataframes is because of Google API errors during geocoding (62 of them, evidently).

In [53]:
non_geocoded_data = non_geocoded_data.drop(["latitude", "longitude"], axis = 1)

In [54]:
non_geocoded_data.to_csv('non_geocoded_data.csv')

In [69]:
re_geocoded_data = pd.read_csv('re_geocoded_data.csv')

In [65]:
test = geocoded_data.append(re_geocoded_data)

In [72]:
test = pd.concat([geocoded_data, re_geocoded_data])

In [74]:
len(test)

2438

In [90]:
test.to_csv('master_data.csv')

In [92]:
len(df[df.state=='AK'])

31

In [229]:
geocoded_without_counts = pd.read_csv('static/data/master_data.csv')

In [230]:
geocoded_without_counts = geocoded_without_counts.sort_values(by = 'location', ascending = True)

In [189]:
df = pd.read_csv('static/data/data.csv')

In [191]:
df = df.sort_values(by = 'location', ascending = True)

In [197]:
counts = pd.Series(df['location'].value_counts())

In [198]:
counts = pd.DataFrame(counts)

In [200]:
counts = counts.reset_index()

In [214]:
counts = counts.rename(columns = {'index' : 'location', 'location' : 'fatality_count'})

In [216]:
counts = counts.sort_values(by = 'location', ascending = True)

In [220]:
geocoded_without_counts['fatality_count'] = counts['fatality_count']

In [224]:
counts[counts['location'] == 'Los Angeles, CA']

,location,fatality_count
0,"Los Angeles, CA",69


In [222]:
geocoded_without_counts[geocoded_without_counts['location'] == 'Los Angeles, CA']

,Unnamed: 0,location,latitude,longitude,fatality_count
32,40,"Los Angeles, CA",34.052234,-118.243685,14


In [236]:
join = pd.merge(geocoded_without_counts, counts, on=['location'])

In [237]:
join[join.location == 'Los Angeles, CA']

,Unnamed: 0,location,latitude,longitude,fatality_count
1266,40,"Los Angeles, CA",34.052234,-118.243685,69


In [239]:
join = join.drop('Unnamed: 0', axis = 1)

In [242]:
join.to_csv('geocoded_locations_with_fatality_counts.csv')

In [244]:
 = df[df.date.str.contains('2015')]
test = df[df.date.str.contains('2017')]      
test = df[df.date.str.contains('2017')]      
test = df[df.date.str.contains('2017')]      
test = df[df.date.str.contains('2017')]      
test = df[df.date.str.contains('2017')]      

In [246]:
len(test)

986

In [247]:
df.date.unique()

array(['2017-12-26', '2017-06-02', '2018-07-28', ..., '2015-04-03',
       '2018-03-26', '2015-06-24'], dtype=object)

In [250]:
years = ['2015', '2016', '2017', '2018', '2019']
for i in years:
    date_pull = df[df.date.str.contains(i)]
    date_pull = date_pull.drop(['Unnamed: 0', 'id'], axis = 1)
    date_pull.to_csv(i+'.csv')

In [251]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4519 entries, 2928 to 252
Data columns (total 16 columns):
Unnamed: 0                 4519 non-null int64
id                         4519 non-null int64
name                       4519 non-null object
date                       4519 non-null object
manner_of_death            4519 non-null object
armed                      4271 non-null object
age                        4341 non-null float64
gender                     4515 non-null object
race                       4116 non-null object
city                       4519 non-null object
state                      4519 non-null object
signs_of_mental_illness    4519 non-null bool
threat_level               4519 non-null object
flee                       4339 non-null object
body_camera                4519 non-null bool
location                   4519 non-null object
dtypes: bool(2), float64(1), int64(2), object(11)
memory usage: 538.4+ KB


In [255]:
len(df[df.armed == 'unarmed'])

289

In [268]:
len(df[df.signs_of_mental_illness == True]) / len(df) * 100

23.58928966585528

In [259]:
df.threat_level.unique()

array(['other', 'attack', 'undetermined'], dtype=object)

In [269]:
(len(df[df.threat_level == 'other']) + len(df[df.threat_level == 'undetermined'])) / len(df) * 100

36.60101792431954

In [266]:
(len(df[df.flee == 'Not fleeing']) / len(df)) * 100

63.99645939367117

In [264]:
df.flee.unique()

array(['Other', 'Not fleeing', 'Foot', 'Car', nan], dtype=object)

In [1]:
import pandas as pd

In [2]:
fatalities = pd.read_csv('static/data/data.csv')

In [9]:
geocoded_locations = pd.read_csv('static/data/master_data.csv')

In [10]:
join = pd.merge(fatalities, geocoded_locations, on=['location'])

In [11]:
join

,Unnamed: 0_x,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,location,Unnamed: 0_y,latitude,longitude
0,0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False,"Shelton, WA",0,47.215094,-123.100707
1,1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False,"Aloha, OR",1,45.494284,-122.867045
2,747,890,Phyllis Ilene Jepsen,2015-10-02,shot,knife,55.0,F,W,Aloha,OR,True,other,Not fleeing,False,"Aloha, OR",1,45.494284,-122.867045
3,2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False,"Wichita, KS",2,37.687176,-97.330053
4,634,765,Nicholas Garner,2015-08-22,shot,NaN,26.0,M,W,Wichita,KS,False,attack,Car,False,"Wichita, KS",2,37.687176,-97.330053
5,1647,1843,Caleb J. Douglas,2016-09-01,shot,gun,18.0,M,W,Wichita,KS,False,other,Car,True,"Wichita, KS",2,37.687176,-97.330053
6,2190,2437,Kevin C. Perry,2017-03-18,shot,gun,25.0,M,W,Wichita,KS,False,attack,Not fleeing,False,"Wichita, KS",2,37.687176,-97.330053
7,2565,2848,Jose Ortiz,2017-08-07,shot,knife,29.0,M,H,Wichita,KS,False,other,Not fleeing,False,"Wichita, KS",2,37.687176,-97.330053
8,2931,3250,Andrew Finch,2017-12-27,shot,unarmed,28.0,M,W,Wichita,KS,False,other,Not fleeing,False,"Wichita, KS",2,37.687176,-97.330053
9,3965,4374,Geoffrey Morris,2019-01-10,shot,gun and vehicle,29.0,M,W,Wichita,KS,False,attack,Car,False,"Wichita, KS",2,37.687176,-97.330053


In [12]:
join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4519 entries, 0 to 4518
Data columns (total 19 columns):
Unnamed: 0_x               4519 non-null int64
id                         4519 non-null int64
name                       4519 non-null object
date                       4519 non-null object
manner_of_death            4519 non-null object
armed                      4271 non-null object
age                        4341 non-null float64
gender                     4515 non-null object
race                       4116 non-null object
city                       4519 non-null object
state                      4519 non-null object
signs_of_mental_illness    4519 non-null bool
threat_level               4519 non-null object
flee                       4339 non-null object
body_camera                4519 non-null bool
location                   4519 non-null object
Unnamed: 0_y               4519 non-null int64
latitude                   4519 non-null float64
longitude                  4519 no

In [13]:
join = join.drop(['Unnamed: 0_x', 'Unnamed: 0_y', 'id'], axis = 1)

In [15]:
join.to_csv('fatalities_geocoded.csv')

In [16]:
len(join)

4519

In [17]:
len(fatalities)

4519